In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from math import sqrt
from sympy.solvers import solve
from sympy import Symbol
from pprint import pprint

In [2]:
df_sww = pd.read_csv("SWW.csv")
df_frate = pd.read_csv("Frate.csv")
df_profiles = pd.read_csv("Profiles.csv")
bucketSize = 0.005 # m3

In [3]:
df_frate["Time to fill 5L bucket (s)"].apply(lambda x: bucketSize/x)

0    0.001340
1    0.001348
2    0.001196
Name: Time to fill 5L bucket (s), dtype: float64

In [4]:
# CONSTANTS
n = 0.01
B = 0.065
# Q = df_frate["Time to fill 5L bucket (s)"].apply(lambda x: bucketSize/x).mean()
Q = 0.001196
g = 9.81 # m/s^2
q = Q/B

In [5]:
df_sww

,Trial,Mean Depth (m),Wave Time (s),Distance (m)
0,1,0.026,8.5,4.038
1,2,0.046,6.3,4.038
2,3,0.054,4.7,4.038


In [6]:
def shallowCelerity(d, g=9.81):
    return sqrt(g*d)

In [7]:
def getErrors(actual, theoretical):
    return abs((actual-theoretical)/theoretical)*100

In [8]:
df_sww["Mean Depth (m)"]

0    0.026
1    0.046
2    0.054
Name: Mean Depth (m), dtype: float64

In [9]:
df_sww["Propagation Speed (m/s)"] = df_sww["Mean Depth (m)"].apply(shallowCelerity)

In [10]:
df_sww["Propagation Speed Actual (m/s)"] = df_sww.apply(lambda rows: rows["Distance (m)"]/rows["Wave Time (s)"], axis=1)

In [11]:
df_sww["Errors (%)"] = df_sww.apply(lambda rows: getErrors(rows["Propagation Speed Actual (m/s)"], rows["Propagation Speed (m/s)"]), axis=1)

In [12]:
df_sww

,Trial,Mean Depth (m),Wave Time (s),Distance (m),Propagation Speed (m/s),Propagation Speed Actual (m/s),Errors (%)
0,1,0.026,8.5,4.038,0.505035,0.475059,5.935400
1,2,0.046,6.3,4.038,0.671759,0.640952,4.585947
2,3,0.054,4.7,4.038,0.727832,0.859149,18.042140


In [13]:
df_profiles

,Condition,Value,Flume 1,Flume 2
0,1,slope (%),0.500,0.500
1,1,depth (m),0.035,0.038
2,2,slope (%),2.000,2.000
3,2,depth (m),0.024,0.014
4,3,slope (%),0.500,2.000
5,3,depth (m),0.022,0.021
6,4,slope (%),2.000,0.500
7,4,depth (m),0.028,0.024


In [14]:
cond1 = {"Condition": 1, "So": (0.005, 0.005), "y": (0.035, 0.038)}
cond2 = {"Condition": 2, "So": (0.02, 0.02), "y": (0.024, 0.014)}
cond3 = {"Condition": 3, "So": (0.005, 0.02), "y":(0.022, 0.021)}
cond4 = {"Condition": 4, "So": (0.02, 0.005), "y": (0.028, 0.024)}

conditions = [cond1, cond2, cond3, cond4]

In [15]:
def getRh(yn, b):
    return (b*yn) / (b+2*yn)

In [16]:
def getC(Rh):
    return (Rh**(1/6)/n)

In [17]:
# function of B, n, Q, So
def getYo(So):
    yo = Symbol("yo")
    normal_eqn = ((yo*B)**(5/3)/(2*yo+B)**(2/3)) - (n*Q)/sqrt(So)
    return solve(normal_eqn, yo)[0]

In [18]:
# uniform flow velocity
def getFr(cond):
    yns = []
    velocities = []
    velocitiesNorm = []
    froudeNumbers = []
    froudeNumbersYn = []
#     velocitiesChezy = []
#     froudeNumbersChezy = []
    for y, slope in zip(cond["y"], cond["So"]):
        yn = getYo(slope)
        rh = getRh(yn, B)
        C = getC(rh)
        velocity = Q/(y*B)
        velocityNorm = Q/(yn*B)
        froudeNumber = velocity/sqrt(g*y)
        froudeNumberYn = velocityNorm/sqrt(g*yn)
        yns.append(yn)
        velocities.append(velocity)
        velocitiesNorm.append(velocityNorm)
        froudeNumbers.append(froudeNumber)
        froudeNumbersYn.append(froudeNumberYn)
#         velocityChezy = C*sqrt(rh*slope)
#         velocitiesChezy.append(velocityChezy)
#         froudeNumberChezy = velocityChezy/sqrt(g*yn)
#         froudeNumbersChezy.append(froudeNumberChezy)
    cond["yn"] = tuple(yns)
    cond["v"] = tuple(velocities)
    cond["v0"] = tuple(velocitiesNorm)
    cond["Fr"] = tuple(froudeNumbers)
    cond["FrYn"] = tuple(froudeNumbersYn)
#     cond["vC"] = tuple(velocitiesChezy)
#     cond["FrC"] = tuple(froudeNumbersChezy)

In [19]:
for condition in conditions:
    getFr(condition)

In [20]:
for condition in conditions:
    pprint(condition)

{'Condition': 1,
 'Fr': (0.8971834480861082, 0.7930634307012617),
 'FrYn': (0.779419049559500, 0.779419049559500),
 'So': (0.005, 0.005),
 'v': (0.5257142857142857, 0.4842105263157895),
 'v0': (0.478640695430140, 0.478640695430140),
 'y': (0.035, 0.038),
 'yn': (0.0384421971965097, 0.0384421971965097)}
{'Condition': 2,
 'Fr': (1.5800340884831183, 3.5464289686944213),
 'FrYn': (1.68546340324189, 1.68546340324189),
 'So': (0.02, 0.02),
 'v': (0.7666666666666666, 1.3142857142857143),
 'v0': (0.800402607584072, 0.800402607584072),
 'y': (0.024, 0.014),
 'yn': (0.0229884308542402, 0.0229884308542402)}
{'Condition': 3,
 'Fr': (1.8003183084212, 1.930431418294691),
 'FrYn': (0.779419049559500, 1.68546340324189),
 'So': (0.005, 0.02),
 'v': (0.8363636363636363, 0.8761904761904761),
 'v0': (0.478640695430140, 0.800402607584072),
 'y': (0.022, 0.021),
 'yn': (0.0384421971965097, 0.0229884308542402)}
{'Condition': 4,
 'Fr': (1.2538519863801196, 1.5800340884831183),
 'FrYn': (1.68546340324189, 0.77

In [21]:
cond4

{'Condition': 4,
 'So': (0.02, 0.005),
 'y': (0.028, 0.024),
 'yn': (0.0229884308542402, 0.0384421971965097),
 'v': (0.6571428571428571, 0.7666666666666666),
 'v0': (0.800402607584072, 0.478640695430140),
 'Fr': (1.2538519863801196, 1.5800340884831183),
 'FrYn': (1.68546340324189, 0.779419049559500)}

In [22]:
# theoretical yc
def cbrt(x):
    if 0 <=x: return x**(1./3.)
    return -(-x)**(1./3.)

yc = cbrt((q**2/g))
yc

0.03255783712440861